# M-SAWI

This notebook calculates the Metropolitan Simulatenous Accessibility Walking Index (M-SAWI) for a territory. This index is the consolidation of every SAWI value of each origins of the territory. The index is defined as:
$$
\text{M-SAWI} = \sum_i \text{SAWI}_i*pp_i
$$
where:
- $\text{SAWI}_i$ denotes the simultaneous accessibility value of origin $i$ within the territory.
- $pp_i$ is the population proportion of origin $i$.

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
# Import the names of the territories
l_names = pd.read_csv('../data/input/table/mpios_names.txt',header=None)
l_names = list(l_names[0])
l_names.remove('bogota-621')

In [ ]:
blocks_norm = gpd.read_file('../data/output/shape/accessibility/indexes/SAWI/SAWI_all_territories_norm.shp')

info = pd.DataFrame(index=l_names,columns=['p_(h)','A_(km2)','rho_(h/km2)','n_blocks'])
m = l_names[0]
for m in l_names:
    blocks_m = blocks_norm[blocks_norm['territory']==m]
    info.loc[m,'p_(h)'] = blocks_m['p_(h)'].sum()
    perimeter = gpd.read_file(f'../data/output/shape/perimeters/{m}_perimeter.shp')
    perimeter = perimeter.to_crs('epsg:32618')
    perimeter['A'] = perimeter.area/1e6
    info.loc[m,'A_(km2)'] = perimeter['A'].sum()
    info.loc[m,'rho_(h/km2)'] = info.loc[m,'p_(h)']/info.loc[m,'A_(km2)']
    info.loc[m,'n_blocks'] = len(blocks_m)
    info.loc[m,'M-SAWI_EQU'] = (blocks_m['SAWI_EQU']*blocks_m['pp']).sum()
    info.loc[m,'M-SAWI_PCA'] = (blocks_m['SAWI_PCA']*blocks_m['pp']).sum()
    info.loc[m,'M-SAWI_AHP'] = (blocks_m['SAWI_AHP']*blocks_m['pp']).sum()
info['p_(h)'] = info['p_(h)'].astype(int)
info['A_(km2)'] = info['A_(km2)'].astype(float)
info['rho_(h/km2)'] = info['rho_(h/km2)'].astype(float)
info['n_blocks'] = info['n_blocks'].astype(int)
info = info.reset_index().rename(columns={'index':'territory'})
info.to_csv('../data/output/table/accessibility_index/indexes/M-SAWI/M-SAWI_all_territories_norm.txt',index=False)